In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.7.0


Data load & pre-processing function

In [5]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)
    
    # keras library to_categorical -> one hot encoding
    # 7 -> [0,0,0,0,0,0,1,0,0]
    
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Create network -> think about which function to use   
1.Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()  # shape 을 펼친다.

def dense(label_dim, weight_init) :   #fully connected layer code
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :   # activation function
    return tf.keras.layers.Activation(tf.keras.activations.relu)

#lab 10 - 3   #rate  = 0.3 node 30% off
def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

#lab 10 - 4
def batch_norm():
    return tf.keras.layers.BatchNormalization()

Make model

In [7]:
class create_model_class(tf.keras.Model):    #class 로 짤시 keras.model 상속 필수
    def __init__(self, label_dim):           # label_dim  : Use for final out put
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()   # 정규 분포 

        self.model = tf.keras.Sequential()     # sequential : list datastructure
        self.model.add(flatten())     # FC layer 를 쓰기 위해 변환
                    # [N, 28, 28, 1]  -> [N,784]   //784 = 28^2
                    #Conv 이용시 flatten 과정 필요없다
            
            # http://taewan.kim/post/cnn/
        
        for i in range(2):
            # shape : [N,784] ->[N,256] ->[N,256]  :  channel : 256
            self.model.add(dense(256, weight_init))
            self.model.add(relu())
            #drop out code 10-3
            self.model.add(dropout(rate = 0.5))
            
        self.model.add(dense(label_dim, weight_init))  #[N,256] -> [N,10] 

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [8]:
# fuction
def create_model_function(label_dim) :
    
    weight_init = tf.keras.initializers.RandomNormal()
    #weight_init = tf.keras.initializers.he_uniform()  # lab 10-2
    #weight_init = tf.keras.initializers.glorot_uniform()  # lab 10-2
    
    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))    #layer
        model.add(batch_norm())               #norm  Batch
        model.add(relu())                      # activation
        
        #model.add(relu())
        #model.add(dropout(rate=0.5))
        
    model.add(dense(label_dim, weight_init))

    return model

Define loss & calculate = > performance function

1. loss function
2. accuracy
3. grad


In [21]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) # training =True == use dropout
    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True)) 
    #categorical_crossentropy : 다중분류 loss function 
    #categorical_crossentropy 내부에서 softmax연산이 수행되지 않는 것으로 보입니다.
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logit_v2(logits=logits,labels=labels))
    
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    # armax output : logits,label : 가장 큰 숫자 위치 반환 
    
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    
    #type cast --- boolean -> float : to calculate accuracy
    # [T,F,T] - > [1,0,1] : accuracy : 2/3
    
    return accuracy

def grad(model, images, labels):   # model weight gradient 
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Define data & hyper-parameter

In [14]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()
# train 6 만 , test 1만장


""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10


train_flag = True # >>>?????


# 이미지와 같은 데이터가 큰것은 size 만큼 데이터를 읽는다. , update 빈도 조절
#  drop_remainder=True : 마지막 남은 데이터를 drop 할거야?


""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()
    # suffle  # 들어가는 data 크기 보다 큰 10만
    # prefetch 배치 사이즈 만큼 학습시 미리 메모리에 
    #batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

11501568/11490434 [==============================] - 1s 0us/step


In [15]:
ds = tf.data.Dataset.range(8) 
ds = ds.batch(3, drop_remainder=True)
list(ds.as_numpy_iterator())
# 0,1,2,3,4,5,6,7   -> 6,7 batch 출력 없어짐
# https://teddylee777.github.io/tensorflow/dataset-batch-window

[array([0, 1, 2]), array([3, 4, 5])]

Define model & optimizer & writer

In [17]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

#Adaptive Moment Estimation (Adam)
#Adagrad + RMSProp
# 과거의 기울기 변화량 참고 stepsize change +mometum ?
#https://onevision.tistory.com/entry/Optimizer-%EC%9D%98-%EC%A2%85%EB%A5%98%EC%99%80-%ED%8A%B9%EC%84%B1-Momentum-RMSProp-Adam
#https://dalpo0814.tistory.com/29


In [22]:
# check point function
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir


""" Writer for log"""
checkpoint_dir = 'checkpoints'   
logs_dir = 'logs'
model_dir = 'nn_relu'
checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
# check point for network in case of restart poin

check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phase

In [23]:
if train_flag :
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                
                """ training sector """
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.2826, train_loss: 2.17763019, train_accuracy: 0.2812, test_Accuracy: 0.2282
Epoch: [ 0] [    1/  468] time: 0.4014, train_loss: 2.12462521, train_accuracy: 0.4531, test_Accuracy: 0.3953
Epoch: [ 0] [    2/  468] time: 0.5168, train_loss: 2.05373907, train_accuracy: 0.6094, test_Accuracy: 0.5147
Epoch: [ 0] [    3/  468] time: 0.6377, train_loss: 1.98109674, train_accuracy: 0.5547, test_Accuracy: 0.5439
Epoch: [ 0] [    4/  468] time: 0.7506, train_loss: 1.88976622, train_accuracy: 0.5625, test_Accuracy: 0.5862
Epoch: [ 0] [    5/  468] time: 0.8617, train_loss: 1.80364251, train_accuracy: 0.6172, test_Accuracy: 0.6185
Epoch: [ 0] [    6/  468] time: 0.9679, train_loss: 1.74465132, train_accuracy: 0.6562, test_Accuracy: 0.6523
Epoch: [ 0] [    7/  468] time: 1.0820, train_loss: 1.69676518, train_accuracy: 0.6172, test_Accuracy: 0.6868
Epoch: [ 0] [    8/  468] time: 1.1967,

Epoch: [ 0] [   74/  468] time: 8.5181, train_loss: 0.49226847, train_accuracy: 0.8828, test_Accuracy: 0.9114
Epoch: [ 0] [   75/  468] time: 8.6249, train_loss: 0.42109126, train_accuracy: 0.8750, test_Accuracy: 0.9107
Epoch: [ 0] [   76/  468] time: 8.7299, train_loss: 0.37879121, train_accuracy: 0.8984, test_Accuracy: 0.9078
Epoch: [ 0] [   77/  468] time: 8.8378, train_loss: 0.33845270, train_accuracy: 0.8984, test_Accuracy: 0.9050
Epoch: [ 0] [   78/  468] time: 8.9440, train_loss: 0.33631060, train_accuracy: 0.8984, test_Accuracy: 0.9062
Epoch: [ 0] [   79/  468] time: 9.0549, train_loss: 0.45797050, train_accuracy: 0.8672, test_Accuracy: 0.9060
Epoch: [ 0] [   80/  468] time: 9.1593, train_loss: 0.40497893, train_accuracy: 0.8672, test_Accuracy: 0.9070
Epoch: [ 0] [   81/  468] time: 9.2655, train_loss: 0.32845205, train_accuracy: 0.9141, test_Accuracy: 0.9092
Epoch: [ 0] [   82/  468] time: 9.3709, train_loss: 0.43510503, train_accuracy: 0.9297, test_Accuracy: 0.9120
Epoch: [ 0

Epoch: [ 0] [  148/  468] time: 16.7699, train_loss: 0.29384267, train_accuracy: 0.9375, test_Accuracy: 0.9272
Epoch: [ 0] [  149/  468] time: 16.8810, train_loss: 0.20343155, train_accuracy: 0.9141, test_Accuracy: 0.9310
Epoch: [ 0] [  150/  468] time: 16.9925, train_loss: 0.19941986, train_accuracy: 0.9531, test_Accuracy: 0.9319
Epoch: [ 0] [  151/  468] time: 17.1013, train_loss: 0.14495575, train_accuracy: 0.9766, test_Accuracy: 0.9328
Epoch: [ 0] [  152/  468] time: 17.2187, train_loss: 0.20723531, train_accuracy: 0.9453, test_Accuracy: 0.9309
Epoch: [ 0] [  153/  468] time: 17.3382, train_loss: 0.17896351, train_accuracy: 0.9453, test_Accuracy: 0.9300
Epoch: [ 0] [  154/  468] time: 17.4544, train_loss: 0.24680421, train_accuracy: 0.9375, test_Accuracy: 0.9278
Epoch: [ 0] [  155/  468] time: 17.5629, train_loss: 0.22901358, train_accuracy: 0.9531, test_Accuracy: 0.9270
Epoch: [ 0] [  156/  468] time: 17.6718, train_loss: 0.34882465, train_accuracy: 0.8906, test_Accuracy: 0.9309
E

Epoch: [ 0] [  222/  468] time: 25.0074, train_loss: 0.25886613, train_accuracy: 0.9453, test_Accuracy: 0.9348
Epoch: [ 0] [  223/  468] time: 25.1220, train_loss: 0.25983721, train_accuracy: 0.9141, test_Accuracy: 0.9349
Epoch: [ 0] [  224/  468] time: 25.2310, train_loss: 0.25851792, train_accuracy: 0.9297, test_Accuracy: 0.9355
Epoch: [ 0] [  225/  468] time: 25.3390, train_loss: 0.17940551, train_accuracy: 0.9297, test_Accuracy: 0.9355
Epoch: [ 0] [  226/  468] time: 25.4492, train_loss: 0.14626011, train_accuracy: 0.9766, test_Accuracy: 0.9367
Epoch: [ 0] [  227/  468] time: 25.5605, train_loss: 0.15918431, train_accuracy: 0.9531, test_Accuracy: 0.9386
Epoch: [ 0] [  228/  468] time: 25.6758, train_loss: 0.23328927, train_accuracy: 0.9297, test_Accuracy: 0.9400
Epoch: [ 0] [  229/  468] time: 25.7861, train_loss: 0.16637155, train_accuracy: 0.9453, test_Accuracy: 0.9403
Epoch: [ 0] [  230/  468] time: 25.8987, train_loss: 0.19115064, train_accuracy: 0.9688, test_Accuracy: 0.9392
E

Epoch: [ 0] [  296/  468] time: 33.2868, train_loss: 0.16370484, train_accuracy: 0.9531, test_Accuracy: 0.9526
Epoch: [ 0] [  297/  468] time: 33.4010, train_loss: 0.14274886, train_accuracy: 0.9609, test_Accuracy: 0.9520
Epoch: [ 0] [  298/  468] time: 33.5111, train_loss: 0.26120809, train_accuracy: 0.9453, test_Accuracy: 0.9509
Epoch: [ 0] [  299/  468] time: 33.6248, train_loss: 0.13360450, train_accuracy: 0.9688, test_Accuracy: 0.9497
Epoch: [ 0] [  300/  468] time: 33.7400, train_loss: 0.12785882, train_accuracy: 0.9766, test_Accuracy: 0.9490
Epoch: [ 0] [  301/  468] time: 33.8514, train_loss: 0.16377230, train_accuracy: 0.9609, test_Accuracy: 0.9490
Epoch: [ 0] [  302/  468] time: 33.9662, train_loss: 0.11800472, train_accuracy: 0.9766, test_Accuracy: 0.9506
Epoch: [ 0] [  303/  468] time: 34.0839, train_loss: 0.15855597, train_accuracy: 0.9531, test_Accuracy: 0.9507
Epoch: [ 0] [  304/  468] time: 34.2027, train_loss: 0.14138451, train_accuracy: 0.9609, test_Accuracy: 0.9516
E

...

Epoch: [ 0] [ 1777/  468] time: 196.8311, train_loss: 0.03108388, train_accuracy: 0.9922, test_Accuracy: 0.9783
Epoch: [ 0] [ 1778/  468] time: 196.9404, train_loss: 0.01745498, train_accuracy: 0.9922, test_Accuracy: 0.9778
Epoch: [ 0] [ 1779/  468] time: 197.0501, train_loss: 0.03230129, train_accuracy: 0.9922, test_Accuracy: 0.9770
Epoch: [ 0] [ 1780/  468] time: 197.1571, train_loss: 0.04089606, train_accuracy: 0.9922, test_Accuracy: 0.9765
Epoch: [ 0] [ 1781/  468] time: 197.2636, train_loss: 0.05055424, train_accuracy: 0.9844, test_Accuracy: 0.9758
Epoch: [ 0] [ 1782/  468] time: 197.3701, train_loss: 0.02804993, train_accuracy: 0.9922, test_Accuracy: 0.9750
Epoch: [ 0] [ 1783/  468] time: 197.4799, train_loss: 0.04711439, train_accuracy: 0.9922, test_Accuracy: 0.9756
Epoch: [ 0] [ 1784/  468] time: 197.5915, train_loss: 0.07252643, train_accuracy: 0.9688, test_Accuracy: 0.9762
Epoch: [ 0] [ 1785/  468] time: 197.6973, train_loss: 0.02641166, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [ 1851/  468] time: 204.8693, train_loss: 0.03524761, train_accuracy: 0.9922, test_Accuracy: 0.9769
Epoch: [ 0] [ 1852/  468] time: 204.9755, train_loss: 0.01393718, train_accuracy: 1.0000, test_Accuracy: 0.9778
Epoch: [ 0] [ 1853/  468] time: 205.0829, train_loss: 0.02268186, train_accuracy: 0.9922, test_Accuracy: 0.9781
Epoch: [ 0] [ 1854/  468] time: 205.1901, train_loss: 0.03827070, train_accuracy: 0.9766, test_Accuracy: 0.9787
Epoch: [ 0] [ 1855/  468] time: 205.3024, train_loss: 0.04725306, train_accuracy: 0.9766, test_Accuracy: 0.9780
Epoch: [ 0] [ 1856/  468] time: 205.4090, train_loss: 0.02737289, train_accuracy: 0.9844, test_Accuracy: 0.9772
Epoch: [ 0] [ 1857/  468] time: 205.5135, train_loss: 0.02839435, train_accuracy: 0.9922, test_Accuracy: 0.9769
Epoch: [ 0] [ 1858/  468] time: 205.6215, train_loss: 0.02787735, train_accuracy: 0.9922, test_Accuracy: 0.9766
Epoch: [ 0] [ 1859/  468] time: 205.7306, train_loss: 0.05504000, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [ 1925/  468] time: 213.2124, train_loss: 0.01491802, train_accuracy: 1.0000, test_Accuracy: 0.9797
Epoch: [ 0] [ 1926/  468] time: 213.3246, train_loss: 0.02636041, train_accuracy: 0.9844, test_Accuracy: 0.9795
Epoch: [ 0] [ 1927/  468] time: 213.4402, train_loss: 0.02748735, train_accuracy: 0.9922, test_Accuracy: 0.9791
Epoch: [ 0] [ 1928/  468] time: 213.5563, train_loss: 0.01521931, train_accuracy: 1.0000, test_Accuracy: 0.9798
Epoch: [ 0] [ 1929/  468] time: 213.6694, train_loss: 0.03407283, train_accuracy: 0.9922, test_Accuracy: 0.9796
Epoch: [ 0] [ 1930/  468] time: 213.7975, train_loss: 0.04137354, train_accuracy: 0.9844, test_Accuracy: 0.9792
Epoch: [ 0] [ 1931/  468] time: 213.9075, train_loss: 0.00844887, train_accuracy: 1.0000, test_Accuracy: 0.9788
Epoch: [ 0] [ 1932/  468] time: 214.0169, train_loss: 0.01425505, train_accuracy: 1.0000, test_Accuracy: 0.9783
Epoch: [ 0] [ 1933/  468] time: 214.1298, train_loss: 0.00822629, train_accuracy: 1.0000, test_Accuracy:

Epoch: [ 0] [ 1999/  468] time: 221.8353, train_loss: 0.01100683, train_accuracy: 1.0000, test_Accuracy: 0.9788
Epoch: [ 0] [ 2000/  468] time: 221.9538, train_loss: 0.05964348, train_accuracy: 0.9844, test_Accuracy: 0.9784
Epoch: [ 0] [ 2001/  468] time: 222.0743, train_loss: 0.01896445, train_accuracy: 0.9922, test_Accuracy: 0.9780
Epoch: [ 0] [ 2002/  468] time: 222.1946, train_loss: 0.01714441, train_accuracy: 0.9922, test_Accuracy: 0.9769
Epoch: [ 0] [ 2003/  468] time: 222.3114, train_loss: 0.03572810, train_accuracy: 0.9844, test_Accuracy: 0.9769
Epoch: [ 0] [ 2004/  468] time: 222.4313, train_loss: 0.00953657, train_accuracy: 0.9922, test_Accuracy: 0.9765
Epoch: [ 0] [ 2005/  468] time: 222.5541, train_loss: 0.04394791, train_accuracy: 0.9844, test_Accuracy: 0.9766
Epoch: [ 0] [ 2006/  468] time: 222.6705, train_loss: 0.04676276, train_accuracy: 0.9766, test_Accuracy: 0.9765
Epoch: [ 0] [ 2007/  468] time: 222.7843, train_loss: 0.00492206, train_accuracy: 1.0000, test_Accuracy:

KeyboardInterrupt: 